In [13]:
# # uncomment this to install backtrader
import sys
!{sys.executable} -m pip install backtrader

In [15]:
import pandas as pd
import matplotlib.pyplot as plt

import backtrader as bt
from datetime import datetime

In [16]:
class firstStrategy(bt.Strategy):
    
    def __init__(self):
        self.rsi = bt.indicators.RSI_SMA(self.data.close, period=25)
        
    def next(self):
        if not self.position:
            if self.rsi < 30:
                self.buy(size=100)
        else:
            if self.rsi > 70:
                self.sell(size=100)

In [17]:
# https://www.backtrader.com/blog/2019-06-13-buy-and-hold/buy-and-hold/
class SimpleBuyAndHold(bt.Strategy):
    def start(self):
        self.val_start = self.broker.get_cash()  # keep the starting cash
        self.size = 0

    def nextstart(self):
        # Buy all the available cash
#         self.size = int(self.broker.get_cash() / self.data)
#         print("Size bought:", self.size)
#         self.buy(size=self.size)
#         self.order_target_value(target=self.broker.get_cash())  # doesn't work
        self.order_target_percent(target=0.99)

    def stop(self):
        # calculate the actual returns
        print("Position:", self.getposition().size)
        self.roi = (self.broker.get_value() / self.val_start) - 1.0
        print('ROI:        {:.2f}%'.format(100.0 * self.roi))        

In [18]:
class SMACross(bt.Strategy):
    
    def __init__(self):
        pfast = 5
        pslow = 25
        sma1 = bt.ind.SMA(period=pfast)
        sma2 = bt.ind.SMA(period=pslow)
        self.crossover = bt.ind.CrossOver(sma1, sma2)
    
    def next(self):
        if not self.position:
            if self.crossover > 0:
                self.order_target_percent(target=0.99) # all in
        elif self.crossover < 0:
            self.order_target_size(target=0)
        

In [21]:
# from https://backtest-rookies.com/2017/06/02/getting-started-first-script/

# set starting cash
startcash = 1000000

cerebro = bt.Cerebro()

# add strategy here
cerebro.addstrategy(SMACross)


# Get SPY data from Yahoo Finance
data = bt.feeds.YahooFinanceData(dataname='AAPL',
                                 fromdate = datetime(2017, 1, 1),
                                 todate = datetime(2020, 9, 30)
                                )
cerebro.adddata(data)
cerebro.broker.setcash(startcash)

print('Starting Portfolio Value: $%.2f' % cerebro.broker.getvalue())

# start backtest here
cerebro.run()

#Get final portfolio Value
portvalue = cerebro.broker.getvalue()
pnl = portvalue - startcash

#Print out the final result
print('Final Portfolio Value: $%.2f' % cerebro.broker.getvalue())
print('P/L: ${}'.format(pnl))

#Finally plot the end results
cerebro.plot()

Starting Portfolio Value: $1000000.00
Final Portfolio Value: $2909296.67
P/L: $1909296.67


ImportError: cannot import name 'warnings' from 'matplotlib.dates' (/home/aryaman/.local/lib/python3.8/site-packages/matplotlib/dates.py)

In [ ]:
# buy and hold P/L: $576803.3700000001